In [ ]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment.
# For the code to do so you can search “mount” in code snippets or use the code given below.
# Our entire drive contents are now mounted on colab at the location “/gdrive”.
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
!pip install vecstack

In [ ]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from collections import Counter #for Smote,

import warnings
warnings.filterwarnings("ignore")


In [ ]:
trainfile = r'/gdrive/My Drive/Colab Notebooks/RevisedHomesiteTrain1.csv'
train_data = pd.read_csv(trainfile)




testfile = r'/gdrive/My Drive/Colab Notebooks/week3_bank/RevisedHomesiteTest1_upd.csv'
test_data = pd.read_csv(testfile)




print(train_data.shape)
print(test_data.shape)
print(train_data.head())

(65000, 596)
(173836, 595)
   CoverageField11A  CoverageField11B  CoverageField1A  CoverageField1B  \
0                 2                 1               17               23   
1                 5                 9                6                8   
2                 4                 6                7               12   
3                15                23                3                2   
4                 4                 6                8               13   

   CoverageField2A  CoverageField2B  CoverageField3A  CoverageField3B  \
0               17               23               15               22   
1                6                8                5                7   
2                7               12                6               10   
3                3                2                2                2   
4                8               13                7               11   

   CoverageField4A  CoverageField4B  ...  PropertyField38_N  \
0               16  

In [ ]:
# Seperate Target column from Train Data
TrainCols = list(train_data.columns.values)
TestCols = list(test_data.columns.values)
print(TrainCols)
print(TestCols)
X_train = train_data[TrainCols[0:len(TrainCols)-1]].copy()
y_train = train_data[['QuoteConversion_Flag']].copy()
print(X_train.shape)
print(y_train.shape)
X_test = test_data
print(X_test.shape)

['CoverageField11A', 'CoverageField11B', 'CoverageField1A', 'CoverageField1B', 'CoverageField2A', 'CoverageField2B', 'CoverageField3A', 'CoverageField3B', 'CoverageField4A', 'CoverageField4B', 'CoverageField5A', 'CoverageField5B', 'CoverageField6A', 'CoverageField6B', 'Field10', 'Field11', 'Field7', 'Field8', 'Field9', 'GeographicField10A', 'GeographicField10B', 'GeographicField11A', 'GeographicField11B', 'GeographicField12A', 'GeographicField12B', 'GeographicField13A', 'GeographicField13B', 'GeographicField14A', 'GeographicField14B', 'GeographicField15A', 'GeographicField15B', 'GeographicField16A', 'GeographicField16B', 'GeographicField17A', 'GeographicField17B', 'GeographicField18A', 'GeographicField18B', 'GeographicField19A', 'GeographicField19B', 'GeographicField1A', 'GeographicField1B', 'GeographicField20A', 'GeographicField20B', 'GeographicField21A', 'GeographicField21B', 'GeographicField22A', 'GeographicField22B', 'GeographicField23A', 'GeographicField23B', 'GeographicField24A',

In [ ]:

print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(y_train))
sm = SMOTE (sampling_strategy=0.40)
#sm = SMOTE (sampling_strategy=0.50)
#sm = SMOTE (sampling_strategy=0.60)
X_res, y_res = sm.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res))


___________________________________________________________________
SMOTE

Original dataset shape Counter({'QuoteConversion_Flag': 1})
Resampled dataset shape Counter({'QuoteConversion_Flag': 1})


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
from sklearn.svm import LinearSVC
linsvm = LinearSVC(max_iter=300)
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()


In [ ]:
from vecstack import stacking
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Define the base models
models = [
    LinearSVC(random_state=42),
    RandomForestClassifier(random_state=42),
    DecisionTreeClassifier(random_state=42)
]

# Initialize the stacking ensemble
S_train, S_test = stacking(models, X_res, y_res, X_test,
                           regression=False,
                           mode='oof_pred_bag',
                           needs_proba=False,
                           save_dir=None,
                           metric=accuracy_score,
                           n_folds=4,
                           stratified=True,
                           shuffle=True,
                           random_state=0,
                           verbose=2)

# Define the parameter grid here
param_grid = {
    'n_estimators': [100, 200]
}

# Initialize the final estimator for the stacked ensemble
stacked_model_final = RandomForestClassifier(random_state=42)

# Initialize RandomizedSearchCV with the final estimator
random_search = RandomizedSearchCV(estimator=stacked_model_final,
                                   param_distributions=param_grid,
                                   n_iter=10,
                                   cv=3,
                                   scoring='accuracy',
                                   n_jobs=-1,
                                   random_state=42)

# Fit the RandomizedSearchCV to the stacked training data (S_train)
random_search.fit(S_train, y_res)

# Output the best parameters and score
print('Best parameters found:', random_search.best_params_)
print('Best cross-validation score:', random_search.best_score_)

# Make predictions with the best model on the stacked test set (S_test)
best_model = random_search.best_estimator_
predictions = best_model.predict(S_test)

# Since you don't have y_test, we estimate performance using cross-validation
# Perform cross-validation on the stacked training data (S_train)
cv_scores = cross_val_score(best_model, S_train, y_res, cv=5, scoring='accuracy')
print('Cross-validated accuracy scores:', cv_scores)
print('Mean CV accuracy:', cv_scores.mean())
print('Standard deviation in CV accuracy:', cv_scores.std())


task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [LinearSVC]
    fold  0:  [0.54672517]
    fold  1:  [0.54702568]
    fold  2:  [0.36612851]
    fold  3:  [0.71204898]
    ----
    MEAN:     [0.54298208] + [0.12236336]
    FULL:     [0.54298214]

model  1:     [RandomForestClassifier]
    fold  0:  [0.91570508]
    fold  1:  [0.91672987]
    fold  2:  [0.92046809]
    fold  3:  [0.91559216]
    ----
    MEAN:     [0.91712380] + [0.00198105]
    FULL:     [0.91712378]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.89132672]
    fold  1:  [0.89419222]
    fold  2:  [0.88947882]
    fold  3:  [0.88958717]
    ----
    MEAN:     [0.89114623] + [0.00190536]
    FULL:     [0.89114624]

Best parameters found: {'n_estimators': 100}
Best cross-validation score: 0.9171237793398616
Cross-validated accuracy scores: [0.87228279 0.87248595 0.88338864 0.9789381  0.97853176]
Mean CV accuracy: 0.9171

In [ ]:
import pandas as pd

# Assuming S_test is a numpy array with the predicted probabilities or classes
# If S_test contains probabilities and you need to convert these to a binary flag, you'd typically do:
# predicted_class = (S_test[:, 1] > 0.5).astype(int)  # Assuming the second column is the probability for class 1

# If S_test already contains classes, you just need the predicted class column, for example:
predicted_class = S_test[:, 0]  # or whichever column is appropriate

# Now create a DataFrame with the predicted classes
predicted_df = pd.DataFrame(predicted_class, columns=['QuoteConversion_Flag'])

# Concatenate the "QuoteNumber" column with the predicted "TARGET" values
result = pd.concat([test_data['QuoteNumber'], predicted_df], axis=1)

# Save the result to a CSV file
result.to_csv('/gdrive/My Drive/Colab Notebooks/week3_bank/homesite_res.csv', index=False)

# Read and display the CSV file
res = pd.read_csv('/gdrive/My Drive/Colab Notebooks/week3_bank/homesite_res.csv')
res.tail()


,QuoteNumber,QuoteConversion_Flag
173831,434570,1
173832,434573,1
173833,434574,1
173834,434575,1
173835,434589,1
